In [140]:
from sentence_transformers import SentenceTransformer
import numpy as np
import requests 

In [141]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

Create the embedding for this user question:


In [142]:
user_question = "I just discovered the course. Can I still join it?"


In [143]:
user_question_vec = embedding_model.encode(user_question)

user_question_vec[0]

0.07822265

## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

In [144]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [145]:
documents[0:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [146]:
filtered_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(filtered_documents)

375

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 


In [147]:
# def create_embeddings(documents):
#     embeddings = []
#     for doc in documents:
#         qa_text = f'{doc["question"]} {doc["text"]}'
#         embedding = embedding_model.encode(qa_text)
#         embeddings.append(embedding)

#     X = np.array(embeddings)
#     return X

In [148]:
embeddings = []


for doc in filtered_documents:
    qa_text = f'{doc["question"]} {doc["text"]}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)

print(X.shape)

(375, 768)



## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


```python
scores = X.dot(v)
```

What's the highest score in the results?

- 65.0 
- 6.5
- **0.65**
- 0.065


In [149]:
v = user_question_vec
# Compute scores using the dot product
scores = X.dot(v)

# Find the highest score
highest_score = np.max(scores)

print(highest_score)

0.6506574


This means that there is an element in the matrix from Q2 which has a similarity score of 0.65 to the question from Q1 ("can i still join the course?")



## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

- The code basically takes the dot product of the embeddings with the `v_query`. 
- it then sorts the results by the scores and returns the `num_results`. Finally it returns the top n results.
- We can refine it by first using `argpartition` and fine-sorting using `argsort` (second step needed or not?)

In [150]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argpartition(-scores, num_results)[:num_results]
        idx = idx[np.argsort(-scores[idx])]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
vector_search_res = search_engine.search(v, num_results=5)
vector_search_res

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c


## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

In [151]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [152]:
ground_truth[0:3]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

In [153]:
# Function to compute hit-rate
def compute_hit_rate(search_engine, ground_truth, num_results=5):
    hits = 0
    total_queries = len(ground_truth)

    for item in ground_truth:
        query = item['question']
        true_doc_id = item['document']

        # Compute the embedding for the query
        v_query = embedding_model.encode(query)

        # Get the top results
        results = search_engine.search(v_query, num_results=num_results)

        # Check if the true document ID is in the top results
        if any(result['id'] == true_doc_id for result in results):
            hits += 1

    return hits / total_queries


hit_rate = compute_hit_rate(search_engine, ground_truth, num_results=5)
hit_rate

0.9398907103825137

In [154]:
print(f'Hit-rate: {hit_rate:.2f}')

Hit-rate: 0.94



## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?


#### Step 2: Create Embeddings using Pretrained Models
(see above)

#### Step 3: Setup ElasticSearch connection 

In [155]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '509561d09229', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KXeqb_dqSiWWCOGYTUYD6A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Step 4: Create Mappings and Index

In [156]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "text"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: add documents to index

In [157]:
for i, d in enumerate(filtered_documents):
    d['text_vector'] = embeddings[i].tolist()

In [158]:
for doc in filtered_documents:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [159]:
query = {
    "field": "text_vector",
    "query_vector": user_question_vec,
    "k": 5,
    "num_candidates": 10000, 
}

In [160]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"][0]["_id"]

'c8bzrJABqX7xbSVQ9_Ae'

## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

In [177]:
item = ground_truth[0]
query = item['question']
# print(query)
true_doc_id = item['document']
# print(true_doc_id)
# Compute the embedding for the query
v_query = embedding_model.encode(query)


query_json = {
"field": "text_vector",
"query_vector": v_query,
"k": 5,
"num_candidates": 10000, 
}

res = es_client.search(index=index_name, knn=query_json, source=["text", "section", "question", "course", "id"])

hits = res["hits"]["hits"]
result_ids = [hit["_source"]["id"] for hit in hits[0:5]]
# print(result_ids)

[id == true_doc_id for id in result_ids]

# if any(result['id'] == true_doc_id for result in results):


# [result_ids == true_doc_id]

# print(hits)
# [hit["id"] for hit in hits[0:5]]
# print([res["hits"]["hits"])
# Get the top results
# results = es_client.search(index=index_name, knn=v_query, source=["text", "section", "question", "course"])
# print(results)

# result_ids = results["hits"]["hits"][0]["_id"]

# # Check if the true document ID is in the top results
# if any(result_ids == true_doc_id):
#     hits += 1

[False, False, False, False, False]

In [170]:
[hit["_source"]["id"] for hit in hits[0:5]]

['0a278fb2', '39fda9f0', 'ee58a693', '4930aa19', '67e2fd13']

In [133]:
true_doc_id

'0227b872'

In [179]:
# Function to compute hit-rate
def compute_hit_rate_elastic(ground_truth, num_results=5):
    hits_counter = 0
    total_queries = len(ground_truth)

    for item in ground_truth:
        query = item['question']
        true_doc_id = item['document']

        # Compute the embedding for the query
        v_query = embedding_model.encode(query)

        query_json = {
        "field": "text_vector",
        "query_vector": v_query,
        "k": 5,
        "num_candidates": 10000, 
}

        # Get the top results
        results = es_client.search(index=index_name, knn=query_json, source=["text", "section", "question", "course", "id"])

        hits = results["hits"]["hits"]

        result_ids = [hit["_source"]["id"] for hit in hits[0:num_results]]

        # Check if the true document ID is in the top results
        if any(id == true_doc_id for id in result_ids):
            hits_counter += 1

    return hits_counter / total_queries

hit_rate = compute_hit_rate_elastic(ground_truth, num_results=5)
hit_rate

0.9398907103825137